In [31]:
import torch

In [32]:
if torch.cuda.is_available():
    device = torch.device('cuda', 0)
else:
    device = torch.device('cpu')

print(device)

cuda:0


In [33]:
nperseg, nstride = 4, 2

ts = torch.rand(16, device=device)
reshape = []
ts = ts[None, None, None, :]
num_segs = (ts.shape[-1] - nperseg) // nstride + 1
stop = (num_segs - 1) * nstride + nperseg
ts = ts[:stop]

print(num_segs, stop)
print(ts)

class unfold(torch.nn.Module):
    def __init__(self):
        super(unfold, self).__init__()
        self.uf = torch.nn.Unfold(
            kernel_size=(1,num_segs),
            dilation=(1,nstride),
            padding=0,
        )

    def forward(self, input):
        x = self.uf(input)
        return x

UF = unfold()
print(UF)
ts_Uf = UF(ts)

7 16
tensor([[[[0.1479, 0.5787, 0.0304, 0.4525, 0.0743, 0.2433, 0.3736, 0.3488,
           0.2965, 0.4961, 0.2199, 0.9754, 0.7900, 0.1528, 0.0338, 0.9799]]]],
       device='cuda:0')
unfold(
  (uf): Unfold(kernel_size=(1, 7), dilation=(1, 2), padding=0, stride=1)
)


In [34]:
print(ts_Uf)
ts_Uf_mean = ts_Uf.mean(axis=-1, keepdims=True)
print(ts_Uf_mean)
ts_diff = ts_Uf - ts_Uf_mean
print(ts_diff)

tensor([[[0.1479, 0.5787, 0.0304, 0.4525],
         [0.0304, 0.4525, 0.0743, 0.2433],
         [0.0743, 0.2433, 0.3736, 0.3488],
         [0.3736, 0.3488, 0.2965, 0.4961],
         [0.2965, 0.4961, 0.2199, 0.9754],
         [0.2199, 0.9754, 0.7900, 0.1528],
         [0.7900, 0.1528, 0.0338, 0.9799]]], device='cuda:0')
tensor([[[0.3024],
         [0.2001],
         [0.2600],
         [0.3787],
         [0.4970],
         [0.5345],
         [0.4891]]], device='cuda:0')
tensor([[[-0.1544,  0.2763, -0.2720,  0.1501],
         [-0.1697,  0.2524, -0.1258,  0.0432],
         [-0.1856, -0.0167,  0.1136,  0.0888],
         [-0.0052, -0.0300, -0.0822,  0.1174],
         [-0.2005, -0.0009, -0.2771,  0.4784],
         [-0.3146,  0.4409,  0.2554, -0.3817],
         [ 0.3009, -0.3363, -0.4553,  0.4908]]], device='cuda:0')


In [36]:
reshape = []

#fft = torch.fft.rfft(ts_Uf, axis=-1).abs() ** 2
fft_diff = torch.fft.rfft(ts_diff, axis=-1).abs() ** 2
print(fft)

if nperseg % 2:
    fft[..., 1:] *= 2
else:
    fft[..., 1:-1] *= 2

print(fft)

print(reshape)
reshape += [num_segs, -1]
print(reshape)

fft = fft.reshape(*reshape)
print(fft)

tensor([[[2.0759e+00, 1.6063e+01, 1.3038e+00],
         [2.3551e+00, 1.0228e+01, 7.9630e-01],
         [1.3892e+00, 5.2623e+01, 2.1430e-01],
         [1.9772e+00, 7.6096e+01, 1.7798e-01],
         [3.1757e+00, 2.4449e+01, 6.7358e-02],
         [1.7089e+00, 4.4527e+01, 3.2134e-01],
         [1.3242e+00, 2.8178e+01, 1.6367e-01]]], device='cuda:0')
tensor([[[2.0759e+00, 3.2126e+01, 1.3038e+00],
         [2.3551e+00, 2.0456e+01, 7.9630e-01],
         [1.3892e+00, 1.0525e+02, 2.1430e-01],
         [1.9772e+00, 1.5219e+02, 1.7798e-01],
         [3.1757e+00, 4.8897e+01, 6.7358e-02],
         [1.7089e+00, 8.9055e+01, 3.2134e-01],
         [1.3242e+00, 5.6356e+01, 1.6367e-01]]], device='cuda:0')
[]
[7, -1]
tensor([[2.0759e+00, 3.2126e+01, 1.3038e+00],
        [2.3551e+00, 2.0456e+01, 7.9630e-01],
        [1.3892e+00, 1.0525e+02, 2.1430e-01],
        [1.9772e+00, 1.5219e+02, 1.7798e-01],
        [3.1757e+00, 4.8897e+01, 6.7358e-02],
        [1.7089e+00, 8.9055e+01, 3.2134e-01],
        [1.3242e+

In [51]:
fft.shape
fft.mean(axis=-2)

tensor([ 2.0009, 72.0467,  0.4350], device='cuda:0')